In [1]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy as np
import pandas as pd
#from xgboost import XGBClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
# season_data[season_data.Season == 2017]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
71241,2017,11,1104,70,1157,53,H,0,21,48,...,23,10,16,17,26,7,13,2,2,22
71242,2017,11,1107,87,1336,81,A,0,30,61,...,25,18,22,6,20,9,12,7,8,23
71243,2017,11,1112,65,1277,63,N,0,20,52,...,25,5,9,12,22,15,18,4,1,23
71244,2017,11,1113,88,1340,70,H,0,33,67,...,27,12,15,9,24,14,14,5,1,20
71245,2017,11,1116,92,1236,83,H,0,35,66,...,28,24,40,21,24,12,12,5,2,24
71246,2017,11,1120,83,1316,66,H,0,25,60,...,20,16,24,14,28,10,28,2,2,30
71247,2017,11,1123,85,1387,64,A,0,30,58,...,16,18,28,13,21,8,16,7,4,22
71248,2017,11,1124,76,1331,61,H,0,24,54,...,13,10,14,9,24,13,14,3,0,19
71249,2017,11,1127,68,1165,62,H,0,23,58,...,29,14,20,9,34,9,12,2,4,19
71250,2017,11,1137,76,1264,64,H,0,25,52,...,22,12,19,19,26,11,24,10,2,26


In [5]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)
exported_pipeline=LogisticRegression()
exported_pipeline.fit(np.array(X), np.array(Y))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
# Create id to team name mappings
teamNames = pd.read_csv('./ncaa-data/Teams.csv')
def teamIDNameMapping():
    team_id_map = {}
    for index, row in teamNames.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    
    return team_id_map

team_id_map = teamIDNameMapping()

In [8]:
def WinChanceMapping(year):
    
    submission_data = []
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    tourney_teams = tourney_seeds[tourney_seeds['Season'] == year]['Team']

    stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
    
    tourney_teams.sort_values()
    
    for team_1 in tourney_teams:
        for team_2 in tourney_teams:
            if team_1 != team_2:
                prediction = mm_predictor.predict_winner(team_1, team_2, exported_pipeline, 2017, stat_fields)
                label = '2017_' + str(team_1) + '_' + str(team_2)
                submission_data.append([label, prediction[0][0]])
    
    
    teamTeamWinChanceMap = {}
    for line in submission_data:
        title = line[0].split('_')
        team1 = team_id_map[int(title[1])]
        team2 = team_id_map[int(title[2])]
        winRate = str(round(line[1]* 100,2))
        if team1 not in teamTeamWinChanceMap:
            teamTeamWinChanceMap[team1] = {}
        if float(winRate) >= 50:
            if team1 in teamTeamWinChanceMap and team2 in teamTeamWinChanceMap[team1] and teamTeamWinChanceMap[team1][team2] > float(winRate):
                continue
            if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
                # update teamTeammap[team2][team1] to remove that key
                teamTeamWinChanceMap[team2].pop(team1, None)
            teamTeamWinChanceMap[team1][team2] = float(winRate)
            
    return teamTeamWinChanceMap

In [9]:
# print("Writing %d results." % len(submission_data))
# with open('./ncaa-data/submission.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['id', 'pred'])
#     writer.writerows(submission_data)

chanceMap = WinChanceMapping(2017)

In [10]:
def filterTourneyDataByYear(year):

    tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
    tourney_data = tourney_data[tourney_data.Season == year]
    
    return tourney_data

def getWinnersList(tourney_data):
    winners = []

    tourney_data_grouped = tourney_data[4:].groupby('WTeamID').size().reset_index(name='NumWins')
    for index, row in tourney_data_grouped.iterrows():
        teamName = team_id_map[row['WTeamID']]
        wins = row['NumWins']
        if len(winners) == 0:
            winners.append([])
        winners[0].append(teamName)
        if wins > 1:
            if len(winners) == 1:
                winners.append([])
            winners[1].append(teamName)
        if wins > 2:
            if len(winners) == 2:
                winners.append([])
            winners[2].append(teamName)
        if wins > 3:
            if len(winners) == 3:
                winners.append([])
            winners[3].append(teamName)
        if wins > 4:
            if len(winners) == 4:
                winners.append([])
            winners[4].append(teamName)
        if wins > 5:
            if len(winners) == 5:
                winners.append([])
            winners[5].append(teamName)
        if wins > 6:
            if len(winners) == 6:
                winners.append([])
            winners[6].append(teamName)
    
    return winners

In [11]:
# Method that takes in season's tournament data, id to name mapping of teams
def calcBracketScore(tourney_data, teamTeamWinChanceMap):
    
    winners = getWinnersList(tourney_data)
    
    # First four rows is 'first four' and not in actual bracket
    firstFour = tourney_data[:4]
    mainTourney = tourney_data[4:]
    
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    
    tourney = [-1] * max(tourney_seeds['Team'])

    # Look at first four teams
    for index, row in firstFour.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
            tourney[index2] = index1
        else:
            tourney[index1] = index2

    # MAIN TOURNAMENT 
    score = 0
    for index, row in mainTourney.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        while tourney[index1] > 0:
            index1 = tourney[index1]
        while tourney[index2] > 0:
            index2 = tourney[index2]
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
            tourney[index2] = index1
            tourney[index1] = tourney[index1] - 1
            if team1Name in winners[abs(tourney[index1]) - 2]:
                score += 2**(abs(tourney[index1]) - 2) * 10
                print(team1Name + ' vs ' + team2Name + ', team 1 wins')
                print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
            else: # Delete else statemenet when done debugging
                print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team1 wins')

        else: #team2 would win
            tourney[index1] = index2
            tourney[index2] = tourney[index2] - 1
            if team2Name in winners[abs(tourney[index2]) - 2]:
                score += 2**(abs(tourney[index2]) - 2) * 10
                print(team1Name + ' vs ' + team2Name + ', team 2 wins')
                print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
            else: # Delete else statement when done debugging
                print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team2 wins')

    return score

In [12]:
data_2017 = filterTourneyDataByYear(2017)
data_2017
print(calcBracketScore(data_2017, chanceMap))

Arizona vs North Dakota, team 1 wins
Score 10
Butler vs Winthrop, team 1 wins
Score 10
Florida vs ETSU, team 1 wins
Score 10
Florida St vs FL Gulf Coast, team 1 wins
Score 10
Gonzaga vs S Dakota St, team 1 wins
Score 10
Iowa St vs Nevada, team 1 wins
Score 10
MTSU vs Minnesota, team 1 wins
Score 10
Northwestern vs Vanderbilt, team 1 wins
Score 10
Notre Dame vs Princeton, team 1 wins
Score 10
Purdue vs Vermont, team 1 wins
Score 10
St Mary's CA vs VA Commonwealth, team 1 wins
Score 10
Villanova vs New Orleans, team 1 wins
Score 10
Virginia vs UNC Wilmington, team 1 wins
Score 10
West Virginia vs Bucknell, team 1 wins
Score 10
Wisconsin vs Virginia Tech, team 1 wins
Score 10
Xavier vs Maryland, team 1 wins
Score 10
Incorrect: Predicted Arkansas vs Seton Hall, team2 wins
Baylor vs New Mexico St, team 1 wins
Score 10
Cincinnati vs Kansas St, team 1 wins
Score 10
Duke vs Troy, team 1 wins
Score 10
Kansas vs UC Davis, team 1 wins
Score 10
Kentucky vs N Kentucky, team 1 wins
Score 10
Louisvil